* dm04 演習テーマ１ 埼玉大学の電力使用量の分析

In [1]:
import os
import sys
import re
import glob
import pandas as pd
import openpyxl # .xlsx ファイルの操作
import xlrd # .xls ファイルの読み込み

In [55]:
data_location = {
    # 日報
    '全体': {'sheetname': '日報', 
           'data': {1:(11,12), 2:(16,12), 3:(13,12), 4:(14,12), 5:(15,12), 6:(16,12), 
                    7:(17,12), 8:(18,12), 9:(19,12), 10:(20,12), 11:(21,12), 12:(22,12), 
                    13:(23,12), 14:(24,12), 15:(25,12), 16:(26,12), 17:(27,12), 18:(28,12), 
                    19:(29,12), 20:(30,12), 21:(31,12), 22:(32,12), 23:(33,12), 24:(34,12),}}, 
    # サブ変電所日報（１）
    '⑪機械棟変電所': {'sheetname': 'サブ変電所日報（１）', 
                'data': {1:(10,12), 2:(11,12), 3:(12,12), 4:(13,12), 5:(14,12), 6:(15,12), 
                         7:(16,12), 8:(17,12), 9:(18,12), 10:(19,12), 11:(20,12), 12:(21,12), 
                         13:(22,12), 14:(23,12), 15:(24,12), 16:(25,12), 17:(26,12), 18:(27,12), 
                         19:(28,12), 20:(29,12), 21:(30,12), 22:(31,12), 23:(32,12), 24:(33,12), }},
    '⑬機能材料変電所': {'sheetname': 'サブ変電所日報（１）', 
                 'data': {1:(10,7), 2:(11,7), 3:(12,7), 4:(13,7), 5:(14,7), 6:(15,7),
                          7:(16,7), 8:(17,7), 9:(18,7), 10:(19,7), 11:(20,7), 12:(21,7),
                          13:(22,7), 14:(23,7), 15:(24,7), 16:(25,7), 17:(26,7), 18:(27,7),
                          19:(28,7), 20:(29,7), 21:(30,7), 22:(31,7), 23:(32,7), 24:(33,7),}},
    # サブ変電所日報（２）
    '⑮建設工学１号館変電所': {'sheetname': 'サブ変電所日報（２）',
                    'data': {1:(10,7), 2:(11,7), 3:(12,7), 4:(13,7), 5:(14,7), 6:(15,7),
                             7:(16,7), 8:(17,7), 9:(18,7), 10:(19,7), 11:(20,7), 12:(21,7),
                             13:(22,7), 14:(23,7), 15:(24,7), 16:(25,7), 17:(26,7), 18:(27,7),
                             19:(28,7), 20:(29,7), 21:(30,7), 22:(31,7), 23:(32,7), 24:(33,7),}},
    '⑭応用化学２号館変電所': {'sheetname': 'サブ変電所日報（２）', 
                'data': {1:(10,12), 2:(11,12), 3:(12,12), 4:(13,12), 5:(14,12), 6:(15,12), 
                         7:(16,12), 8:(17,12), 9:(18,12), 10:(19,12), 11:(20,12), 12:(21,12), 
                         13:(22,12), 14:(23,12), 15:(24,12), 16:(25,12), 17:(26,12), 18:(27,12), 
                         19:(28,12), 20:(29,12), 21:(30,12), 22:(31,12), 23:(32,12), 24:(33,12), }},
    '①本部変電所': {'sheetname': 'サブ変電所日報（２）', 
                'data': {1:(10,16), 2:(11,16), 3:(12,16), 4:(13,16), 5:(14,16), 6:(15,16), 
                         7:(16,16), 8:(17,16), 9:(18,16), 10:(19,16), 11:(20,16), 12:(21,16), 
                         13:(22,16), 14:(23,16), 15:(24,16), 16:(25,16), 17:(26,16), 18:(27,16), 
                         19:(28,16), 20:(29,16), 21:(30,16), 22:(31,16), 23:(32,16), 24:(33,16), }},
    '⑯地域共同変電所': {'sheetname': 'サブ変電所日報（２）', 
                'data': {1:(10,19), 2:(11,19), 3:(12,19), 4:(13,19), 5:(14,19), 6:(15,19), 
                         7:(16,19), 8:(17,19), 9:(18,19), 10:(19,19), 11:(20,19), 12:(21,19), 
                         13:(22,19), 14:(23,19), 15:(24,19), 16:(25,19), 17:(26,19), 18:(27,19), 
                         19:(28,19), 20:(29,19), 21:(30,19), 22:(31,19), 23:(32,19), 24:(33,19), }},
    # サブ変電所日報（３）
    '③教養学部変電所': {'sheetname': 'サブ変電所日報（３）',
                    'data': {1:(10,8), 2:(11,8), 3:(12,8), 4:(13,8), 5:(14,8), 6:(15,8),
                             7:(16,8), 8:(17,8), 9:(18,8), 10:(19,8), 11:(20,8), 12:(21,8),
                             13:(22,8), 14:(23,8), 15:(24,8), 16:(25,8), 17:(26,8), 18:(27,8),
                             19:(28,8), 20:(29,8), 21:(30,8), 22:(31,8), 23:(32,8), 24:(33,8),}},
    # サブ変電所日報（４）
    '⑤理学部３号館変電所': {'sheetname': 'サブ変電所日報（４）',
                    'data': {1:(10,9), 2:(11,9), 3:(12,9), 4:(13,9), 5:(14,9), 6:(15,9),
                             7:(16,9), 8:(17,9), 9:(18,9), 10:(19,9), 11:(20,9), 12:(21,9),
                             13:(22,9), 14:(23,9), 15:(24,9), 16:(25,9), 17:(26,9), 18:(27,9),
                             19:(28,9), 20:(29,9), 21:(30,9), 22:(31,9), 23:(32,9), 24:(33,9),}},
    '⑥理学部２号館変電所': {'sheetname': 'サブ変電所日報（４）', 
                'data': {1:(10,13), 2:(11,13), 3:(12,13), 4:(13,13), 5:(14,13), 6:(15,13), 
                         7:(16,13), 8:(17,13), 9:(18,13), 10:(19,13), 11:(20,13), 12:(21,13), 
                         13:(22,13), 14:(23,13), 15:(24,13), 16:(25,13), 17:(26,13), 18:(27,13), 
                         19:(28,13), 20:(29,13), 21:(30,13), 22:(31,13), 23:(32,13), 24:(33,13), }},
    # サブ変電所日報（５）
    '⑨電気電子１号変電所': {'sheetname': 'サブ変電所日報（５）',
                   'data': {1:(11,2), 2:(16,2), 3:(13,2), 4:(14,2), 5:(15,2), 6:(16,2), 
                            7:(17,2), 8:(18,2), 9:(19,2), 10:(20,2), 11:(21,2), 12:(22,2), 
                            13:(23,2), 14:(24,2), 15:(25,2), 16:(26,2), 17:(27,2), 18:(28,2), 
                            19:(29,2), 20:(30,2), 21:(31,2), 22:(32,2), 23:(33,2), 24:(34,2),}}, 
    '⑩情報工学科棟変電所': {'sheetname': 'サブ変電所日報（５）',
                   'data': {1:(10,8), 2:(11,8), 3:(12,8), 4:(13,8), 5:(14,8), 6:(15,8),
                            7:(16,8), 8:(17,8), 9:(18,8), 10:(19,8), 11:(20,8), 12:(21,8),
                            13:(22,8), 14:(23,8), 15:(24,8), 16:(25,8), 17:(26,8), 18:(27,8),
                            19:(28,8), 20:(29,8), 21:(30,8), 22:(31,8), 23:(32,8), 24:(33,8),}},
    '⑧情報分析センタ－変電所': {'sheetname': 'サブ変電所日報（５）',
                   'data': {1:(10,14), 2:(11,14), 3:(12,14), 4:(13,14), 5:(14,14), 6:(15,14),
                            7:(16,14), 8:(17,14), 9:(18,14), 10:(19,14), 11:(20,14), 12:(21,14),
                            13:(22,14), 14:(23,14), 15:(24,14), 16:(25,14), 17:(26,14), 18:(27,14),
                            19:(28,14), 20:(29,14), 21:(30,14), 22:(31,14), 23:(32,14), 24:(33,14),}},
    # サブ変電所日報（６）
    '②大学会館変電所': {'sheetname': 'サブ変電所日報（６）',
                   'data': {1:(10,8), 2:(11,8), 3:(12,8), 4:(13,8), 5:(14,8), 6:(15,8),
                            7:(16,8), 8:(17,8), 9:(18,8), 10:(19,8), 11:(20,8), 12:(21,8),
                            13:(22,8), 14:(23,8), 15:(24,8), 16:(25,8), 17:(26,8), 18:(27,8),
                            19:(28,8), 20:(29,8), 21:(30,8), 22:(31,8), 23:(32,8), 24:(33,8),}},
    '⑦図書館変電所': {'sheetname': 'サブ変電所日報（６）',
                   'data': {1:(10,19), 2:(11,19), 3:(12,19), 4:(13,19), 5:(14,19), 6:(15,19),
                            7:(16,19), 8:(17,19), 9:(18,19), 10:(19,19), 11:(20,19), 12:(21,19),
                            13:(22,19), 14:(23,19), 15:(24,19), 16:(25,19), 17:(26,19), 18:(27,19),
                            19:(28,19), 20:(29,19), 21:(30,19), 22:(31,19), 23:(32,19), 24:(33,19),}},
    # サブ変電所日報（７）
    '⑳廃液処理施設変電所': {'sheetname': 'サブ変電所日報（７）',
                   'data': {1:(10,4), 2:(11,4), 3:(12,4), 4:(13,4), 5:(14,4), 6:(15,4),
                            7:(16,4), 8:(17,4), 9:(18,4), 10:(19,4), 11:(20,4), 12:(21,4),
                            13:(22,4), 14:(23,4), 15:(24,4), 16:(25,4), 17:(26,4), 18:(27,4),
                            19:(28,4), 20:(29,4), 21:(30,4), 22:(31,4), 23:(32,4), 24:(33,4),}},
    '⑰教育学部変電所': {'sheetname': 'サブ変電所日報（７）', 
                'data': {1:(10,13), 2:(11,13), 3:(12,13), 4:(13,13), 5:(14,13), 6:(15,13), 
                         7:(16,13), 8:(17,13), 9:(18,13), 10:(19,13), 11:(20,13), 12:(21,13), 
                         13:(22,13), 14:(23,13), 15:(24,13), 16:(25,13), 17:(26,13), 18:(27,13), 
                         19:(28,13), 20:(29,13), 21:(30,13), 22:(31,13), 23:(32,13), 24:(33,13), }},
    '⑱体育施設変電所': {'sheetname': 'サブ変電所日報（７）', 
                'data': {1:(10,17), 2:(11,17), 3:(12,17), 4:(13,17), 5:(14,17), 6:(15,17), 
                         7:(16,17), 8:(17,17), 9:(18,17), 10:(19,17), 11:(20,17), 12:(21,17), 
                         13:(22,17), 14:(23,17), 15:(24,17), 16:(25,17), 17:(26,17), 18:(27,17), 
                         19:(28,17), 20:(29,17), 21:(30,17), 22:(31,17), 23:(32,17), 24:(33,17), }},
    # サブ変電所日報（８）
    '④経済学部変電所': {'sheetname': 'サブ変電所日報（８）',
                   'data': {1:(10,5), 2:(11,5), 3:(12,5), 4:(13,5), 5:(14,5), 6:(15,5),
                            7:(16,5), 8:(17,5), 9:(18,5), 10:(19,5), 11:(20,5), 12:(21,5),
                            13:(22,5), 14:(23,5), 15:(24,5), 16:(25,5), 17:(26,5), 18:(27,5),
                            19:(28,5), 20:(29,5), 21:(30,5), 22:(31,5), 23:(32,5), 24:(33,5),}},
    '⑫総合研究棟変電所': {'sheetname': 'サブ変電所日報（８）', 
                'data': {1:(10,13), 2:(11,13), 3:(12,13), 4:(13,13), 5:(14,13), 6:(15,13), 
                         7:(16,13), 8:(17,13), 9:(18,13), 10:(19,13), 11:(20,13), 12:(21,13), 
                         13:(22,13), 14:(23,13), 15:(24,13), 16:(25,13), 17:(26,13), 18:(27,13), 
                         19:(28,13), 20:(29,13), 21:(30,13), 22:(31,13), 23:(32,13), 24:(33,13), }},
                } # 


In [60]:
def get_data_from_excel(excel_files, output_file):
    # 集計先となるエクセルのブックを準備
    output_wb = openpyxl.Workbook()

    for key_1 in data_location:
        # 元データのどのシートのどのデータを処理するか出力
        print('###', key_1, 'in', data_location[key_1]['sheetname'])
        # 出力用のワークブックに、ワークシートを作成して、出力用に準備
        output_wb.create_sheet(key_1)
        output_ws = output_wb[key_1] 
        # A列に項目名を入力
        output_ws.cell(row=1, column=1).value = 'Time'
        for i in range(24):
            output_ws.cell(row=2+i, column=1).value = i + 1
        output_column = 2
        output_ws.cell(row=26, column=1).value = 'Total'
        output_ws.cell(row=27, column=1).value = 'Peak'
        output_ws.cell(row=28, column=1).value = 'Day'
        output_ws.cell(row=29, column=1).value = 'Night'
        output_ws.cell(row=31, column=1).value = 'Max'

        # リスト中のエクセルファイルからデータを取得し、ワークシートに転記
        for excel_file in excel_files:
            # ファイル名から年月日を取得
            m = re.search(r'(202\d)(\d\d)(\d\d).xls', excel_file)
            if m is None:
                print(f'## unsupported file name', excel_file)
                return
            year = int(m.group(1))
            month = int(m.group(2))
            mday = int(m.group(3))

            # リスト中のエクセルファイルを開き、処理対象のワークシートがあるか確認
            wb = xlrd.open_workbook(excel_file)
            sheet_names = wb.sheet_names()
            # print(excel_file, sheet_names) # [デバッグ用] エクセルファイル中のシート一覧を出力
            if data_location[key_1]['sheetname'] not in sheet_names:
                print(f'### {year}/{month}/{mday} : sheet {data_location[key_1]["sheetname"]} is not in {excel_file}')
            else:
                # 出力先シートに、データの年月日と、各時間帯の電力使用量の合計を計算する関数を挿入
                output_ws.cell(row=1, column=output_column).value = f'{year}{month:02d}{mday:02d}'
                output_ws.cell(row=26, column=output_column).value = f'=SUM({output_ws.cell(row=2, column=output_column).coordinate}:{output_ws.cell(row=25, column=output_column).coordinate})'
                output_ws.cell(row=27, column=output_column).value = f'=SUM({output_ws.cell(row=14, column=output_column).coordinate}:{output_ws.cell(row=16, column=output_column).coordinate})'
                output_ws.cell(row=28, column=output_column).value = f'=SUM({output_ws.cell(row=9, column=output_column).coordinate}:{output_ws.cell(row=13, column=output_column).coordinate})' + \
                                                                     f'+SUM({output_ws.cell(row=17, column=output_column).coordinate}:{output_ws.cell(row=23, column=output_column).coordinate})'
                output_ws.cell(row=29, column=output_column).value = f'=SUM({output_ws.cell(row=2, column=output_column).coordinate}:{output_ws.cell(row=8, column=output_column).coordinate})' + \
                                                                     f'+SUM({output_ws.cell(row=24, column=output_column).coordinate}:{output_ws.cell(row=25, column=output_column).coordinate})'
                output_ws.cell(row=31, column=output_column).value = f'=MAX({output_ws.cell(row=2, column=output_column).coordinate}:{output_ws.cell(row=25, column=output_column).coordinate})'
                # リスト中のエクセルファイルからデータを出力先シートに転記
                sheet = wb.sheet_by_name(data_location[key_1]['sheetname'])
                output_row = 2
                for key_2 in data_location[key_1]['data'].keys():
                    _row, _column = data_location[key_1]['data'][key_2] 
                    output_ws.cell(row=output_row, column=output_column).value = sheet.cell(_row, _column).value
                    output_row = output_row + 1
                output_column = output_column + 1
    
    # ワークブックを出力 (空のシートは削除)
    output_wb.remove(output_wb['Sheet'])
    output_wb.save(output_file)

In [64]:
# 電力データを保存したフォルダにあるエクセルファイルのリストを作成
data_folder = r'E:\sudspg\資料\2023_電力使用量\2022_電力データ' # 電力データを保存したフォルダ名
data_files = sorted(glob.glob(os.path.join(data_folder, '202*.xls'))) # エクセルファイル名のリストを取得
print(f'# {len(data_files)} files')
print(data_files[0])

# リスト中のエクセルファイルを集約したエクセルファイルを作成
get_data_from_excel(data_files, 'dm04_sample_data.xlsx')

# 331 files
E:\sudspg\資料\2023_電力使用量\2022_電力データ\20220401.xls
### 全体 in 日報
### ⑪機械棟変電所 in サブ変電所日報（１）
### ⑬機能材料変電所 in サブ変電所日報（１）
### ⑮建設工学１号館変電所 in サブ変電所日報（２）
### ⑭応用化学２号館変電所 in サブ変電所日報（２）
### ①本部変電所 in サブ変電所日報（２）
### ⑯地域共同変電所 in サブ変電所日報（２）
### ③教養学部変電所 in サブ変電所日報（３）
### 2022/4/27 : sheet サブ変電所日報（３） is not in E:\sudspg\資料\2023_電力使用量\2022_電力データ\20220427.xls
### ⑤理学部３号館変電所 in サブ変電所日報（４）
### 2022/4/27 : sheet サブ変電所日報（４） is not in E:\sudspg\資料\2023_電力使用量\2022_電力データ\20220427.xls
### ⑥理学部２号館変電所 in サブ変電所日報（４）
### 2022/4/27 : sheet サブ変電所日報（４） is not in E:\sudspg\資料\2023_電力使用量\2022_電力データ\20220427.xls
### ⑨電気電子１号変電所 in サブ変電所日報（５）
### 2022/4/27 : sheet サブ変電所日報（５） is not in E:\sudspg\資料\2023_電力使用量\2022_電力データ\20220427.xls
### 2022/12/29 : sheet サブ変電所日報（５） is not in E:\sudspg\資料\2023_電力使用量\2022_電力データ\20221229.xls
### ⑩情報工学科棟変電所 in サブ変電所日報（５）
### 2022/4/27 : sheet サブ変電所日報（５） is not in E:\sudspg\資料\2023_電力使用量\2022_電力データ\20220427.xls
### 2022/12/29 : sheet サブ変電所日報（５） is not in E:\sudspg\資料\2023_電力使用量\2022